In [1]:
import numpy as np
import cv2
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import scipy.misc
import pandas as pd

In [2]:
'''
출처
https://github.com/inyl/my_notebook/blob/master/open_cv/image_color_cluster.ipynb
https://www.pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/
'''

'\n출처\nhttps://github.com/inyl/my_notebook/blob/master/open_cv/image_color_cluster.ipynb\nhttps://www.pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/\n'

In [3]:
'''
카페 DF가 다 모인 후에 주석을 풀고 실행
'''
df_cafe = pd.read_csv('final_cafe_info.csv')

In [4]:
df_cafe.head()

,지역,지역구,카페명,주소,위도,경도,Unnamed: 6
0,서서울,은평,스모어,서울 은평구 연서로29길 8-8,37.618952,126.919697,NaN
1,서서울,은평,필라멘트카페,서울 은평구 통일로 883,37.621753,126.919462,NaN
2,서서울,은평,HUGA,서울 은평구 진관2로 19,37.634753,126.919774,NaN
3,서서울,은평,YM Coffee Project,서울 은평구 연서로29길 21-8,37.619095,126.917542,NaN
4,서서울,은평,카페달력,서울 은평구 연서로18길 28-2,37.611930,126.920534,NaN


In [5]:
df_cafe = df_cafe.drop('Unnamed: 6', axis=1)

In [6]:
df_cafe.head()

,지역,지역구,카페명,주소,위도,경도
0,서서울,은평,스모어,서울 은평구 연서로29길 8-8,37.618952,126.919697
1,서서울,은평,필라멘트카페,서울 은평구 통일로 883,37.621753,126.919462
2,서서울,은평,HUGA,서울 은평구 진관2로 19,37.634753,126.919774
3,서서울,은평,YM Coffee Project,서울 은평구 연서로29길 21-8,37.619095,126.917542
4,서서울,은평,카페달력,서울 은평구 연서로18길 28-2,37.611930,126.920534


In [7]:
area_names = {'ws' : ['ep', 'mp', 'sdm'],
              'sw' : ['gs', 'yc', 'ydp', 'gr'],
              'cs' : ['jr', 'jg', 'ys'],
              'ss' : ['dj', 'ga', 'gc'],
              'gn' : ['sc', 'gn'],
              'gb' : ['db', 'gb', 'sb', 'nw'],
              'es' : ['ddm', 'gl', 'sd', 'gj'],
              'se' : ['gd', 'sp']
             }
# area_names['ws'][0]

In [8]:
path_list = [f'{k}_{v}_{i:02}.png'
            for k in area_names.keys() 
            for v in area_names[k] 
            for i in range(1,14)]
path_list[2]

'ws_ep_03.png'

In [9]:
df_cafe['파일명'] = path_list

In [10]:
df_cafe.head()

,지역,지역구,카페명,주소,위도,경도,파일명
0,서서울,은평,스모어,서울 은평구 연서로29길 8-8,37.618952,126.919697,ws_ep_01.png
1,서서울,은평,필라멘트카페,서울 은평구 통일로 883,37.621753,126.919462,ws_ep_02.png
2,서서울,은평,HUGA,서울 은평구 진관2로 19,37.634753,126.919774,ws_ep_03.png
3,서서울,은평,YM Coffee Project,서울 은평구 연서로29길 21-8,37.619095,126.917542,ws_ep_04.png
4,서서울,은평,카페달력,서울 은평구 연서로18길 28-2,37.611930,126.920534,ws_ep_05.png


In [11]:
df_cafe.tail()

,지역,지역구,카페명,주소,위도,경도,파일명
320,동남,송파,카페단지,서울 송파구 백제고분로45길 12,37.509586,127.110133,se_sp_09.png
321,동남,송파,어나더선데이,서울 송파구 송파대로48길 3,37.508446,127.104935,se_sp_10.png
322,동남,송파,벨라스가든,서울 송파구 백제고분로41길 42-19,37.510044,127.106276,se_sp_11.png
323,동남,송파,고고스,서울 송파구 석촌호수로 197,37.509240,127.099154,se_sp_12.png
324,동남,송파,파티세리 도효,서울 송파구 오금로16길 10,37.510768,127.108722,se_sp_13.png


In [12]:
df_cafe.to_csv('final_cafe_info_with_path.csv')

In [13]:
def centroid_histogram(clt):
    '''
    # grab the number of different clusters and create a histogram
    히스토그램 형식으로 색을 반환
    based on the number of pixels assigned to each cluster
    각 클러스터의 픽셀의 숫자를 기반으로 함
    '''
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # normalize the histogram, such that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()  # hist = hist/hist.sum()

    # return the histogram
    return hist

In [14]:
def plot_colors(hist, centroids):
    '''
    initialize the bar chart representing the relative frequency of each of the colors
    각 색의 빈도를 나타내는 바 차트를 초기화
    '''
    bar = np.zeros((50, 300, 3), dtype="uint8")
    startX = 0

    # loop over the percentage of each cluster and the color of each cluster
    for (percent, color) in zip(hist, centroids):
        # plot the relative percentage of each cluster
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                      color.astype("uint8").tolist(), -1)
        startX = endX

    # return the bar chart
    return bar

In [15]:
def image_color_cluster(image_path, k = 5):
    image = cv2.imread(image_path)
    # image의 shape을 찍어보면, height, width, channel 순으로 나옴
    # channel은 RGB를 말함
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # cv에서는 RGB가 아닌 BGR 순으로 나오기 때문에 순서를 RGB로 전환
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    # shape의 0,1번째 즉, height와 width를 통합시킴
    
    clt = KMeans(n_clusters = k)  # 평균 알고리즘 KMeans
    clt.fit(image)

    hist = centroid_histogram(clt)
    bar = plot_colors(hist, clt.cluster_centers_)
    return bar

In [16]:
path_list[268]

'es_gl_09.png'

In [17]:
for index in range(268, len(path_list)):
    result = image_color_cluster('./cafe_image/'+path_list[index])
    scipy.misc.imsave('./cafe_color_result/'+path_list[index], result)

C:\Users\rlath\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
